# Cybershuttle SDK -  Molecular Dynamics
> Define, run, monitor, and analyze molecular dynamics experiments in a HPC-agnostic way.

This notebook shows how users can setup and launch a **NAMD** experiment with replicas, monitor its execution, and run analyses both during and after execution.

## Installing Required Packages

First, install the `airavata-python-sdk-test` package from the pip repository.

In [ ]:
%pip install --upgrade airavata-python-sdk-test

## Importing the SDK

In [ ]:
import airavata_experiments as ae
from airavata_experiments import md

## Authenticating

To authenticate for remote execution, call the `ae.login()` method.
This method will prompt you to enter your credentials and authenticate your session.

In [ ]:
ae.login()

Once authenticated, the `ae.list_runtimes()` function can be called to list HPC resources that the user has access to.

In [ ]:
runtimes = ae.list_runtimes()
ae.display(runtimes)

## Uploading Experiment Files

Drag and drop experiment files onto the workspace that this notebook is run on.

```bash
(sh) $: tree data
data
├── b4pull.pdb
├── b4pull.restart.coor
├── b4pull.restart.vel
├── b4pull.restart.xsc
├── par_all36_water.prm
├── par_all36m_prot.prm
├── pull.conf
├── structure.pdb
└── structure.psf

1 directory, 9 files

```

## Defining a NAMD Experiment

The `md.NAMD.initialize()` is used to define a NAMD experiment.
Here, provide the paths to the `.conf` file, the `.pdb` file, the `.psf` file, any optional files you want to run NAMD on.
You can preview the function definition through auto-completion.

```python
def initialize(
    name: str,
    config_file: str,
    pdb_file: str,
    psf_file: str,
    ffp_files: list[str],
    other_files: list[str] = [],
    parallelism: Literal['CPU', 'GPU'] = "CPU",
    num_replicas: int = 1
) -> Experiment[ExperimentApp]
```

To add replica runs, simply call the `exp.add_replica()` function.
You can call the `add_replica()` function as many times as you want replicas.
Any optional resource constraint can be provided here.

You can also call `ae.display()` to pretty-print the experiment.

In [ ]:
exp = md.NAMD.initialize(
    name="yasith_namd_experiment",
    config_file="data/pull_cpu.conf",
    pdb_file="data/structure.pdb",
    psf_file="data/structure.psf",
    ffp_files=[
      "data/par_all36_water.prm",
      "data/par_all36m_prot.prm"
    ],
    other_files=[
      "data/b4pull.pdb",
      "data/b4pull.restart.coor",
      "data/b4pull.restart.vel",
      "data/b4pull.restart.xsc",
    ],
    parallelism="CPU",
    num_replicas=1,
)
exp.add_replica(*ae.list_runtimes(cluster="login.expanse.sdsc.edu", category="cpu"))
ae.display(exp)

## Creating an Execution Plan

Call the `exp.plan()` function to transform the experiment definition + replicas into a stateful execution plan.

In [ ]:
plan = exp.plan()
ae.display(plan)

## Saving the Plan

A created plan can be saved locally (in JSON) or remotely (in a user-local DB) for later reference.

In [ ]:
plan.save()  # this will save the plan in DB
plan.save_json("plan.json")  # save the plan state locally

## Launching the Plan

A created plan can be launched using the `plan.launch()` function.
Changes to plan states will be automatically saved onto the remote.
However, plan state can also be tracked locally by invoking `plan.save_json()`.

In [ ]:
plan.launch()
plan.save_json("plan.json")

## Checking the Plan Status
The status of a plan can be retrieved by calling `plan.status()`.

In [ ]:
plan.status()

## Loading a Saved Plan

A saved plan can be loaded by calling `ae.plan.load_json(plan_path)` (for local plans) or `ae.plan.load(plan_id)` (for remote plans).

In [ ]:
plan = ae.plan.load_json("plan.json")
plan = ae.plan.load(plan.id)
plan.status()
ae.display(plan)

## Fetching User-Defined Plans

The `ae.plan.query()` function retrieves all plans stored in the remote.

In [ ]:
plans = ae.plan.query()
ae.display(plans)

## Managing Plan Execution

The `plan.stop()` function will stop a currently executing plan.
The `plan.wait_for_completion()` function would block until the plan finishes executing.

In [ ]:
# plan.stop()
plan.wait_for_completion()

## Interacting with Files

The `task` object has several helper functions to perform file operations within its context.

* `task.ls()` - list all remote files (inputs, outputs, logs, etc.)
* `task.upload(<local_path>, <remote_path>)` - upload a local file to remote
* `task.cat(<remote_path>)` - displays contents of a remote file
* `task.download(<remote_path>, <local_path>)` - fetch a remote file to local

In [ ]:
for task in plan.tasks:
    print(task.name, task.pid)
    # display files
    display(task.ls())
    # upload a file
    task.upload("data/sample.txt")
    # preview contents of a file
    display(task.cat("sample.txt"))
    # download a specific file
    task.download("sample.txt", f"./results_{task.name}")
    # download all files
    task.download_all(f"./results_{task.name}")

## Executing Task-Local Code Remotely

The `@task.context()` decorator can be applied on Python functions to run them remotely within the task context.
The functions executed this way has access to the task files, as well as the remote compute resources.

**NOTE**: Currently, remote code execution is only supported for ongoing tasks. In future updates, we will support both ongoing and completed tasks. Stay tuned!

In [ ]:
for index, task in enumerate(plan.tasks):
    @task.context(packages=["numpy", "pandas"])
    def analyze() -> None:
        import numpy as np
        with open("pull.conf", "r") as f:
            data = f.read()
        print("pull.conf has", len(data), "chars")
        print(np.arange(10))
    analyze()